In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=10
N_FOLDS = 5

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Grape",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 5241 files belonging to 5 classes.


In [4]:

len(dataset)

164

In [5]:
# split dataset into train, validation, and test sets
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds
    

In [6]:
train_ds_full, val_ds_full, test_ds_full = get_dataset_partitions_tf(dataset)


In [7]:
# use k-fold cross-validation
fold_scores = []
fold_history = []


In [8]:
for fold in range(N_FOLDS):
    print(f'Fold {fold+1}/{N_FOLDS}')
    
    # split into train and validation datasets
    val_ds = train_ds_full.skip(fold * len(train_ds_full) // N_FOLDS).take(len(train_ds_full) // N_FOLDS)
    train_ds = train_ds_full.concatenate(test_ds_full).concatenate(val_ds).shuffle(buffer_size=len(train_ds_full))
    

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [9]:
# build model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

    

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [11]:
 model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )


In [ ]:
# train model
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)

Epoch 1/10
  9/174 [>.............................] - ETA: 5:01 - loss: 483.2829 - accuracy: 0.1910

In [27]:
# evaluate model
scores = model.evaluate(test_ds_full)
fold_scores.append(scores)
fold_history.append(history)

17/17 [==============================] - 10s 415ms/step - loss: 0.0441 - accuracy: 0.9853


In [28]:
# print results
mean_scores = np.mean(fold_scores, axis=0)
std_scores = np.std(fold_scores, axis=0)
print(f'Mean accuracy: {mean_scores[1]:.2f} +/- {std_scores[1]:.2f}')

Mean accuracy: 0.99 +/- 0.00
